In [12]:
import pandas as pd
from unidecode import unidecode
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, lit, lower, udf

In [4]:
spark = SparkSession.builder.appName('MyApp').getOrCreate()

23/03/19 14:40:17 WARN Utils: Your hostname, MacBook-Air-di-Emanuele.local resolves to a loopback address: 127.0.0.1; using 10.217.14.148 instead (on interface en0)
23/03/19 14:40:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 14:40:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
#wiki = pd.read_csv("../datasets/wiki_movie_plots_deduped.csv")

wiki = spark.read.option("escape", "\"").csv('../datasets/wiki_movie_plots_deduped.csv',header=True, inferSchema=True)
wiki

DataFrame[Release Year: string, Title: string, Origin/Ethnicity: string, Director: string, Cast: string, Genre: string, Wiki Page: string, Plot: string]

In [17]:
#train = pd.read_csv("../datasets/gpt_data/train_plot_awards.csv", sep=',')
train = spark.read.option("escape", "\"").csv("../datasets/gpt_data/train_plot_awards.csv", sep=",", header=True, inferSchema=True)
train

DataFrame[_c0: int, tconst: string, primaryTitle: string, originalTitle: string, startYear: int, endYear: string, runtimeMinutes: string, numVotes: double, label: boolean, plot: string, awards: int]

In [18]:
#train["startYear"].replace(r"\N", None, inplace=True)
#train["startYear"].fillna(train['endYear'], inplace=True)
#train["startYear"] = train["startYear"].astype(int)

train = train.withColumn("startYear", regexp_replace(col("startYear"), r"\\N", str(None)))
train = train.fillna({'startYear': 'endYear'}, subset=['startYear'])
#train = train.select("startYear").fillna(train.select("endYear"))
train = train.withColumn("startYear", col("startYear").cast("int"))

In [19]:
udf_unidecode = udf(lambda x: unidecode(x) if x is not None else None)

In [20]:
#train["primaryTitle"] = train["primaryTitle"].str.lower()
#train['primaryTitle'] = train['primaryTitle'].apply(unidecode)

train = train.withColumn("primaryTitle", lower(col("primaryTitle")))
train = train.withColumn("primaryTitle", udf_unidecode(train["primaryTitle"]))

In [22]:
#wiki["Title"] = wiki["Title"].str.lower()
#wiki.rename({"Title": "primaryTitle"}, axis=1, inplace=True)
#wiki.rename({"Release Year": "startYear"}, axis=1, inplace=True)
#wiki['primaryTitle'] = wiki['primaryTitle'].apply(unidecode)

wiki = wiki.withColumn("Title", lower(col("Title")))
wiki = wiki.withColumnRenamed("Title", "primaryTitle")
wiki = wiki.withColumnRenamed("Release Year", "startYear")
wiki = wiki.withColumn("primaryTitle", udf_unidecode(wiki["primaryTitle"]))

In [23]:
# remove rows where the 'name' column is a duplicate
#wiki = wiki.drop_duplicates(subset=['primaryTitle'], keep='last')

wiki = wiki.dropDuplicates(subset=["primaryTitle"])

In [ ]:
# find the duplicate rows based on all columns
wiki[wiki["primaryTitle"] == "interstellar"]

In [ ]:
pd.merge(train, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

In [25]:
#final_train = pd.merge(train, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

final_train = train.join(wiki.select('primaryTitle', 'Genre', 'startYear'), on=['primaryTitle', 'startYear'], how='left')
final_train

DataFrame[primaryTitle: string, startYear: int, _c0: int, tconst: string, originalTitle: string, endYear: string, runtimeMinutes: string, numVotes: double, label: boolean, plot: string, awards: int, Genre: string]

In [26]:
#final_train.to_csv("../datasets/gpt_data/train_plots_awards_genre_no_duplicates.csv", index=False)

final_train.toPandas().to_csv('../datasets/gpt_data/train_plots_awards_genre_no_duplicates.csv', index=False)

### Validation

In [27]:
# validation = pd.read_csv("../datasets/gpt_data/validation_plots_awards.csv", sep=',')

validation = spark.read.option("escape", "\"").csv("../datasets/gpt_data/validation_plots_awards.csv", sep=",", header=True, inferSchema=True)
validation

DataFrame[Unnamed: 0: int, tconst: string, primaryTitle: string, originalTitle: string, startYear: int, endYear: string, runtimeMinutes: string, numVotes: double, plot: string, awards: int]

In [28]:
#validation["startYear"].replace(r"\N", None, inplace=True)
#validation["startYear"].fillna(validation['endYear'], inplace=True)
#validation["startYear"] = validation["startYear"].astype(int)

validation = validation.withColumn("startYear", regexp_replace(col("startYear"), r"\\N", str(None)))
validation = validation.fillna({'startYear': 'endYear'}, subset=['startYear'])
#validation = validation.select("startYear").fillna(validation.select("endYear"))
validation = validation.withColumn("startYear", col("startYear").cast("int"))

In [29]:
#validation["primaryTitle"] = validation["primaryTitle"].str.lower()
#validation['primaryTitle'] = validation['primaryTitle'].apply(unidecode)

validation = validation.withColumn("primaryTitle", lower(col("primaryTitle")))
validation = validation.withColumn("primaryTitle", udf_unidecode(validation["primaryTitle"]))

In [ ]:
pd.merge(validation, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

In [32]:
#validation_final = pd.merge(validation, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

validation_final = validation.join(wiki.select('primaryTitle', 'Genre', 'startYear'), on=['primaryTitle', 'startYear'], how='left')
validation_final

DataFrame[primaryTitle: string, startYear: int, Unnamed: 0: int, tconst: string, originalTitle: string, endYear: string, runtimeMinutes: string, numVotes: double, plot: string, awards: int, Genre: string]

In [33]:
#validation_final.to_csv("../datasets/gpt_data/validation_plots_awards_genre_no_duplicates.csv", index=False)

validation.toPandas().to_csv("../datasets/gpt_data/validation_plots_awards_genre_no_duplicates.csv", index=False)

### Test data

In [34]:
# test = pd.read_csv("../datasets/gpt_data/test_plots_awards.csv")
test = spark.read.option("escape", "\"").csv("../datasets/gpt_data/test_plots_awards.csv",
                                             sep=",", header=True, inferSchema=True)

In [36]:
#test["startYear"].replace(r"\N", None, inplace=True)
#test["startYear"].fillna(test['endYear'], inplace=True)
#test["startYear"] = test["startYear"].astype(int)
#test["primaryTitle"] = test["primaryTitle"].str.lower()
#test['primaryTitle'] = test['primaryTitle'].apply(unidecode)

test = test.withColumn("startYear", regexp_replace(col("startYear"), r"\\N", str(None)))
test = test.fillna({'startYear': 'endYear'}, subset=['startYear'])
#test = test.select("startYear").fillna(test.select("endYear"))
test = test.withColumn("startYear", col("startYear").cast("int"))
test = test.withColumn("primaryTitle", lower(col("primaryTitle")))
test = test.withColumn("primaryTitle", udf_unidecode(test["primaryTitle"]))

In [37]:
test

DataFrame[Unnamed: 0: int, tconst: string, primaryTitle: string, originalTitle: string, startYear: int, endYear: string, runtimeMinutes: string, numVotes: double, plot: string, awards: int]

In [40]:
#pd.merge(test, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

test.join(wiki.select('primaryTitle', 'Genre', 'startYear'), on=['primaryTitle', 'startYear'], how='left').show()

+--------------------+---------+----------+---------+-------------------+-------+--------------+--------+--------------------+------+----------------+
|        primaryTitle|startYear|Unnamed: 0|   tconst|      originalTitle|endYear|runtimeMinutes|numVotes|                plot|awards|           Genre|
+--------------------+---------+----------+---------+-------------------+-------+--------------+--------+--------------------+------+----------------+
| laugh, clown, laugh|     1928|        67|tt0019074|Laugh, Clown, Laugh|     \N|            73|  1934.0|  Laugh, Clown, L...|     0|       melodrama|
|   the farmer's wife|     1928|        66|tt0018876|               null|     \N|           100|  2741.0|  The Farmer's Wi...|     0| romantic comedy|
|          metropolis|     1927|        46|tt0017136|               null|     \N|           153|168372.0|  Metropolis is a...|     1|            null|
|       mr. thank you|     1936|       157|tt0027307|               null|   1936|            7

In [41]:
#test_final = pd.merge(test, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

test_final = test.join(wiki.select('primaryTitle', 'Genre', 'startYear'), on=['primaryTitle', 'startYear'], how='left')
test_final

DataFrame[primaryTitle: string, startYear: int, Unnamed: 0: int, tconst: string, originalTitle: string, endYear: string, runtimeMinutes: string, numVotes: double, plot: string, awards: int, Genre: string]

In [42]:
#test_final.to_csv("../datasets/gpt_data/test_plots_awards_genre_no_duplicates.csv", index=False)

test_final.toPandas().to_csv('../datasets/gpt_data/test_plots_awards_genre_no_duplicates.csv', index=False)

In [9]:
pd.merge(test, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,plot,awards,Genre
0,22,tt0014972,he who gets slapped,He Who Gets Slapped,1924,\N,95,3654.0,He Who Gets Slapped is a 1924 silent film di...,0,drama
1,23,tt0015016,the iron horse,NaN,1924,\N,150,2136.0,The Iron Horse is a 1924 American silent wes...,0,western
2,26,tt0015174,die nibelungen: kriemhild's revenge,NaN,1924,\N,129,4341.0,The Die Nibelungen: Kriemhild's Revenge is a...,0,NaN
3,28,tt0015214,at 3:25,NaN,1925,1925,59,1724.0,At 3:25 is a drama film about a young woman ...,0,NaN
4,34,tt0015863,go west,NaN,1925,\N,69,4188.0,Go West is a 1925 silent comedy film starrin...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1081,9942,tt9430698,one piece: stampede,NaN,2019,\N,101,5109.0,The Straw Hat Pirates are invited to the Pir...,0,NaN
1082,9943,tt9441638,the big ugly,NaN,2020,\N,106,5780.0,The Big Ugly is a 2020 crime drama set in th...,0,NaN
1083,9948,tt9495690,pagalpanti,Pagalpanti,2019,\N,149,2331.0,The Págálpánti is a 2019 Indian drama film d...,0,NaN
1084,9950,tt9519642,the wedding unplanner,NaN,2020,\N,110,NaN,The Wedding Unplanner is a 2020 romantic com...,0,NaN


In [10]:
test_final = pd.join(test, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")
test_final

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,plot,awards,Genre
0,22,tt0014972,he who gets slapped,He Who Gets Slapped,1924,\N,95,3654.0,He Who Gets Slapped is a 1924 silent film di...,0,drama
1,23,tt0015016,the iron horse,NaN,1924,\N,150,2136.0,The Iron Horse is a 1924 American silent wes...,0,western
2,26,tt0015174,die nibelungen: kriemhild's revenge,NaN,1924,\N,129,4341.0,The Die Nibelungen: Kriemhild's Revenge is a...,0,NaN
3,28,tt0015214,at 3:25,NaN,1925,1925,59,1724.0,At 3:25 is a drama film about a young woman ...,0,NaN
4,34,tt0015863,go west,NaN,1925,\N,69,4188.0,Go West is a 1925 silent comedy film starrin...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1081,9942,tt9430698,one piece: stampede,NaN,2019,\N,101,5109.0,The Straw Hat Pirates are invited to the Pir...,0,NaN
1082,9943,tt9441638,the big ugly,NaN,2020,\N,106,5780.0,The Big Ugly is a 2020 crime drama set in th...,0,NaN
1083,9948,tt9495690,pagalpanti,Pagalpanti,2019,\N,149,2331.0,The Págálpánti is a 2019 Indian drama film d...,0,NaN
1084,9950,tt9519642,the wedding unplanner,NaN,2020,\N,110,NaN,The Wedding Unplanner is a 2020 romantic com...,0,NaN


In [11]:
test_final.to_csv("../datasets/gpt_data/test_plots_awards_genre_no_duplicates.csv", index=False)